In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import oandapyV20.endpoints.forexlabs as labs

In [3]:
orders = pd.concat([
    pd.read_csv('order_book/january.csv'),
    pd.read_csv('order_book/january_missing.csv'),
    pd.read_csv('order_book/february.csv'),
    pd.read_csv('order_book/march.csv'),
])

In [4]:
positions = pd.concat([
    pd.read_csv('position_book/january.csv'),
    pd.read_csv('position_book/february.csv'),
    pd.read_csv('position_book/march.csv'),
])

In [5]:
def hist(api, instrument, start_days, end_days, granularity):

    start_date = (dt.datetime.now()-dt.timedelta(days=start_days)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_date = (dt.datetime.now()-dt.timedelta(days=end_days, hours=2,minutes=4)).strftime('%Y-%m-%dT%H:%M:%SZ')

    params ={
                "from": start_date,
                "to": end_date,
                "granularity":granularity,
            }

    df_list = []
    for r in InstrumentsCandlesFactory(instrument=instrument,params=params):
        api.request(r)
        df = pd.DataFrame(r.response['candles'])
        if(df.empty==False):
            time = df['time']
            volume = pd.DataFrame(df['volume'].apply(pd.Series))
            df = pd.DataFrame(df['mid'].apply(pd.Series))
            df = pd.concat([df,time,volume], axis=1)
            df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
            #df.set_index('time',inplace=True)
            df_list.append(df)
    
    final = pd.concat(df_list)
    
    names = {
        'o': 'open',
        'c': 'close',
        'h': 'high',
        'l': 'low',
        0: 'vol',
        'time': 'time',
    }
    new_names = []
    for column_name in final.columns:
        new_names.append(names[column_name])
    final.columns = new_names
    
    return final

In [6]:
def cal(client, instrument, perdiod):

    
    
    params = {
        "instrument": instrument,
        "period": perdiod
    }
    
    # PERIOD VALUES
    #3600 - 1 hour
    #43200 - 12 hours
    #86400 - 1 day
    #604800 - 1 week
    #2592000 - 1 month
    #7776000 - 3 months
    #15552000 - 6 months
    #31536000 - 1 year
    # http://developer.oanda.com/rest-live/forex-labs/

    r = labs.Calendar(params=params)
    client.request(r)
    
    df = pd.DataFrame.from_dict(r.response, orient='columns')
    
    df['timestamp'] = pd.to_datetime(df['timestamp']*1000000000)
    df = df[['impact', 'timestamp']]
    df.columns = ['impact', 'time']

    return df.groupby('time').sum().reset_index()

In [7]:
def merge(history, calendar):
    return pd.merge(history, calendar, left_on = 'time', right_on = 'time', how='outer')\
                                                                            .set_index('time')\
                                                                            .astype(float)\
                                                                            .fillna(0)

In [8]:
def broaden_impact(df, period):
    df = df.reset_index().sort_values('time').set_index('time')

    for i in range(periods):
        df.loc[
            (df['impact'].shift(-1-i) != 0)
        ,'impact'] = df['impact'].shift(-1-i)
    
    df['impact'] = df['impact'].fillna(0)

    return df[df['low'] != 0]

In [9]:
client = API(access_token='3cf916fab91f55a904250b95078a6851-27cefccf52cf1a0aead8d8385376e890')

In [10]:
granularity_param = 'M5'

In [11]:
calendar = cal(client, 'EUR_USD', 7776000)

In [12]:
history = hist(client, 'EUR_USD', 90, 0, granularity_param)

In [13]:
merged = merge(history, calendar)

In [14]:
merged.head(1)

,open,high,low,close,vol,impact
time,,,,,,
2019-01-04 20:55:00,1.13976,1.1403,1.13963,1.1403,101.0,0.0


In [15]:
orders.head(1)

,Unnamed: 0,time,price,roundedPrice,level_0_l,level_0_s,level_1_up_l,level_1_up_s,level_1_down_l,level_1_down_s,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s
0,0,2019-01-03T15:40:00Z,1.13998,1.14,0.0065,0.0978,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [16]:
positions.head(1)

,Unnamed: 0,time,price,roundedPrice,level_0_l,level_0_s,level_1_up_l,level_1_up_s,level_1_down_l,level_1_down_s,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s
0,0,2019-01-01T00:00:00Z,1.14649,1.1465,0.3569,0.1814,0.272,0.0848,0.272,0.0848,...,0.1463,0.0936,0.0,0.0,0.0,0.0,0,0,0,0


In [17]:
orders = orders.set_index(pd.to_datetime(orders['time'], format="%Y-%m-%dT%H:%M:%SZ"))

In [18]:
ordersNewColumns = []
for column in orders.columns:
    ordersNewColumns.append(column+'_orders')

In [19]:
orders.columns = ordersNewColumns

In [20]:
positions = positions.set_index(pd.to_datetime(positions['time'], format="%Y-%m-%dT%H:%M:%SZ"))

In [21]:
positionsNewColumns = []
for column in positions.columns:
    positionsNewColumns.append(column+'_positions')

In [22]:
positions.columns = positionsNewColumns

In [23]:
base = merged.join(orders).join(positions)\
    .dropna()\
    .drop(['Unnamed: 0_orders', 'Unnamed: 0_positions', 'price_orders',
          'roundedPrice_orders', 'price_positions', 'roundedPrice_positions',
          'time_orders', 'time_positions'], axis = 1)\

In [24]:
base.head(1)

,open,high,low,close,vol,impact,level_0_l_orders,level_0_s_orders,level_1_up_l_orders,level_1_up_s_orders,...,level_3_down_l_positions,level_3_down_s_positions,level_4_up_l_positions,level_4_up_s_positions,level_4_down_l_positions,level_4_down_s_positions,level_5_up_l_positions,level_5_up_s_positions,level_5_down_l_positions,level_5_down_s_positions
time,,,,,,,,,,,,,,,,,,,,,
2019-01-04 21:00:00,1.14035,1.14036,1.14006,1.14006,51.0,0.0,0.0229,0.0589,0.0589,0.1277,...,0.282,0.1763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
len(base)/72 # more less number of days

59.81944444444444

In [26]:
base['ma5'] = base['close'].rolling(5).mean()
base['ma10'] = base['close'].rolling(10).mean()
base = base.dropna()

In [107]:
df_list = []

for date in sorted(set(base.index.date)):
    for startingHour in [6, 7, 8]:
        for openingInterval in [1, 2]:
            for bottomBarrierPips in [0.0060, 0.0040]:
                # TO BE DONE - prepare data with order book
                
                openTradesDF = base[
                    (
                        base.index.date == date
                    ) &
                    (
                        (
                            (base.index.hour >= startingHour) &
                            (base.index.hour < startingHour + openingInterval) &
                            (base.index.minute != 0)
                        ) |
                        (
                            (base.index.hour == startingHour + openingInterval) &
                            (base.index.minute == 0)
                        )
                    )
                ]
                
                firstCandle = base[
                    (base.index.date == date) &
                    (base.index.hour >= startingHour) &
                    (base.index.hour < startingHour + openingInterval) &
                    (base.index.minute == 0)
                ]
                
                if len(openTradesDF) == 3 * openingInterval:
                    openTradePrices = []
                    basePrice = firstCandle.iloc[0]['open']
                    bottomBarrier = basePrice - bottomBarrierPips
                    
                    for i in range(len(openTradesDF)):
                        price = openTradesDF.iloc[i]['open']
                        
                        if openTradePrices == []:
                            if\
                                openTradesDF.iloc[i]['ma5'] < openTradesDF.iloc[i]['ma10'] and\
                                price > bottomBarrier and\
                                price < basePrice:
                                
                                openTradePrices.append(price)
                        else:
                            if\
                                openTradesDF.iloc[i]['ma5'] < openTradesDF.iloc[i]['ma10'] and\
                                price > bottomBarrier and\
                                price < basePrice and\
                                price < min(openTradePrices):
                                
                                openTradePrices.append(price)
                                
                    if openTradePrices != []:
                        for averageMultiplayer in [1.10, 1.15, 1.20]: #
                            
                            weightedPrices = []
                            weights = []
                            for i, price in enumerate(openTradePrices):
                                if i == 0:
                                    weightedPrices.append(price)
                                    weights.append(1)
                                else:
                                    weightedPrices.append(price * averageMultiplayer * i)
                                    weights.append(averageMultiplayer * i)
                            averageOpenPrice = sum(weightedPrices) / sum(weights)
                            numberOfTrades = len(openTradePrices)
                            
                        for slPips in [0.0010, 0.0015]: #[0.0020, 0.0030, 0.0040, 0.0050]:
                            for tpMultiplier in [1.2, 1.3, 1.4]:
                                for endingHour in [11, 12, 13, 14, 15, 16, 17, 18, 19]:
                                    
                                    checkTradesDF = base[
                                        (base.index.date == date) &
                                        (base.index.hour >= startingHour + openingInterval) &
                                        (base.index.hour <= endingHour)
                                    ].iloc[:-2]
                                    
                                    # edge case: one trade which is very low is under sl lvl
                                    slPrice = averageOpenPrice - slPips
                                    tpPrice = averageOpenPrice + (slPips * tpMultiplier)
                                    resultPips = 0.0
                                    
                                    if checkTradesDF.iloc[0]['open'] < bottomBarrier:
                                        resultPips = (bottomBarrier - averageOpenPrice)\
                                                        * numberOfTrades
                                    if checkTradesDF.iloc[0]['open'] < slPrice:
                                        resultPips = (slPrice - averageOpenPrice)\
                                                        * numberOfTrades
                                    else:
                                        closeTradeFlag = False
                                        for candle in checkTradesDF.iloc[1:][['high', 'low']].to_dict('rows'):
                                            if candle['low'] < slPrice:
                                                resultPips = (slPrice - averageOpenPrice)\
                                                        * numberOfTrades
                                                closeTradeFlag = True
                                                break
                                            if candle['high'] > tpPrice:
                                                resultPips = (tpPrice - averageOpenPrice)\
                                                        * numberOfTrades
                                                closeTradeFlag = True
                                                break
                                                
                                    if closeTradeFlag == False:
                                        resultPips = (checkTradesDF.iloc[-1]['open'] - averageOpenPrice)\
                                                        * numberOfTrades
                                            
                                    
                                    
                                    row = {
                                        'date': date,
                                        'startingHour': startingHour,
                                        'openingInterval': openingInterval,
                                        'bottomBarrierPips': bottomBarrierPips,
                                        'averageMultiplayer': averageMultiplayer,
                                        'slPips': slPips,
                                        'tpMultiplier': tpMultiplier,
                                        'endingHour': endingHour,
                                        'resultPips': resultPips,
                                    }
                                    
                                    df_list.append(row)

In [108]:
scores = pd.DataFrame(df_list)

In [109]:
scores.groupby([
    'averageMultiplayer',
    'bottomBarrierPips',
    'endingHour',
    'openingInterval',
    'slPips',
    'startingHour',
    'tpMultiplier',
]).agg({'resultPips': 'sum', 'date': 'count'})\
    .sort_values('resultPips', ascending=False)\
    .head(20)

resultPips  \
averageMultiplayer bottomBarrierPips endingHour openingInterval slPips startingHour tpMultiplier               
1.2                0.004             12         1               0.0015 8            1.4             0.014513   
                   0.006             12         1               0.0015 8            1.4             0.014513   
                   0.004             12         1               0.0015 8            1.3             0.014043   
                   0.006             12         1               0.0015 8            1.3             0.014043   
                                                                                    1.2             0.012393   
                   0.004             12         1               0.0015 8            1.2             0.012393   
                                     11         1               0.0010 6            1.3             0.012114   
                   0.006             11         1               0.0010 6            1.3             0.012114   
                                     14         1               0.0015 8            1.4             0.010897   
                   0.004             14         1               0.0015 8            1.4             0.010897   
                                     11         1               0.0010 6            1.4             0.010454   
                   0.006             11         1               0.0010 6            1.4             0.010454   
                                                                0.0015 6            1.4             0.010332   
                   0.004             11         1               0.0015 6            1.4             0.010332   
                                     14         1               0.0010 8            1.2             0.010199   
                   0.006             14         1               0.0010 8            1.2             0.010199   
                   0.004             11         1               0.0010 6            1.2             0.010014   
                   0.006             11         1               0.0010 6            1.2             0.010014   
                   0.004             13         1               0.0015 8            1.4             0.009597   
                   0.006             13         1               0.0015 8            1.4             0.009597   

                                                                                                  date  
averageMultiplayer bottomBarrierPips endingHour openingInterval slPips startingHour tpMultiplier        
1.2                0.004             12         1               0.0015 8            1.4             26  
                   0.006             12         1               0.0015 8            1.4             26  
                   0.004             12         1               0.0015 8            1.3             26  
                   0.006             12         1               0.0015 8            1.3             26  
                                                                                    1.2             26  
                   0.004             12         1               0.0015 8            1.2             26  
                                     11         1               0.0010 6            1.3             28  
                   0.006             11         1               0.0010 6            1.3             28  
                                     14         1               0.0015 8            1.4             26  
                   0.004             14         1               0.0015 8            1.4             26  
                                     11         1               0.0010 6            1.4             28  
                   0.006             11         1               0.0010 6            1.4             28  
                                                                0.0015 6            1.4             28  
                   0.004             11         1         

In [106]:
scoresBigSL = scores